In [ ]:
import sys 
sys.path.append("../..")
sys.path.append("..")

from importlib import reload
from tqdm import tqdm

import joseph
from joseph.analysis import *
from joseph.visualisation import *
from joseph.utils import *
from joseph.data import *


reload(joseph.analysis)
reload(joseph.visualisation)
reload(joseph.utils)
reload(joseph.data)

from joseph.analysis import *
from joseph.visualisation import *
from joseph.utils import *
from joseph.data import *

# turn torch grad tracking off
torch.set_grad_enabled(False)


model = HookedTransformer.from_pretrained(
    "gpt2-small",
    fold_ln=True,
)
model.set_use_split_qkv_input(True)
model.set_use_attn_result(True)




# path = "../artifacts/sparse_autoencoder_gpt2-small_blocks.8.hook_resid_pre_24576:v9/final_sparse_autoencoder_gpt2-small_blocks.8.hook_resid_pre_24576.pt"
path = "../artifacts/sparse_autoencoder_gpt2-small_blocks.10.hook_resid_pre_24576:v19/final_sparse_autoencoder_gpt2-small_blocks.10.hook_resid_pre_24576.pt"
sparse_autoencoder = SparseAutoencoder.load_from_pretrained(path)

print(sparse_autoencoder.cfg)


# sanity check
text = "Many important transition points in the history of science have been moments when science 'zoomed in.' At these points, we develop a visualization or tool that allows us to see the world in a new level of detail, and a new field of science develops to study the world through this lens."
display(model(text, return_type="loss"))

from sae_training.utils import LMSparseAutoencoderSessionloader
model, sparse_autoencoder, activation_store = LMSparseAutoencoderSessionloader.load_session_from_pretrained(
    path
)

sparse_autoencoder.cfg.use_ghost_grads = False


import webbrowser
from IPython.core.display import display, HTML

path_to_html = "../../feature_dashboards/gpt2-small_blocks.10.hook_resid_pre_24576"
def render_feature_dashboard(feature_id):
    
    path = f"{path_to_html}/data_{feature_id:04}.html"
    
    print(f"Feature {feature_id}")
    if os.path.exists(path):
        # with open(path, "r") as f:
        #     html = f.read()
        #     display(HTML(html))
        webbrowser.open_new_tab("file://" + os.path.abspath(path))
    else:
        print("No HTML file found")
    
    return

# for feature in [100,300,400]:
#     render_feature_dashboard(feature)

In [ ]:

import joseph
reload(joseph.analysis)
from joseph.analysis import *

title = "Brackets"
prompt = "The quick brown fox jumps over the lazy dog (and then the dog jumps over the fox). Then the fox jumps over the dog again (and then the dog jumps over the fox)."
POS_INTEREST = 18

# title = "Maths"
# prompt = "120 + (30 * 2) - 10 = (30 * (53 - 20)) + 10"
# POS_INTEREST = 4

token_df, original_cache, cache_reconstructed_query, feature_acts = eval_prompt([prompt], model, sparse_autoencoder, head_idx_override=7)
filter_cols = ["str_tokens", "unique_token", "context", "batch", "pos", "label", "loss", "loss_diff", "mse_loss", "num_active_features", "explained_variance", "kl_divergence",
            "top_k_features"]
display(token_df[filter_cols].style.background_gradient(
    subset=["loss_diff", "mse_loss","explained_variance", "num_active_features", "kl_divergence"],
    cmap="coolwarm"))



UNIQUE_TOKEN_INTEREST = token_df["unique_token"][POS_INTEREST]
feature_acts_of_interest = feature_acts[POS_INTEREST]
# plot_line_with_top_10_labels(feature_acts_of_interest, "", 25)
# vals, inds = torch.topk(feature_acts_of_interest,39)

top_k_feature_inds = (feature_acts[1:] > 0).sum(dim=0).nonzero().squeeze()

features_acts_by_token_df = pd.DataFrame(
    feature_acts[:,top_k_feature_inds[:]].detach().cpu().T,
    index = [f"feature_{i}" for i in top_k_feature_inds.flatten().tolist()],
    columns = token_df["unique_token"])

# features_acts_by_token_df.sort_values(by=",/12", ascending=False).head(10).style.background_gradient(
#     cmap="coolwarm", axis=0)

# px.imshow(features_acts_by_token_df.sort_values(by=",/12", ascending=False).head(10).T.corr(), color_continuous_midpoint=0, color_continuous_scale="RdBu")
# n_feature_dashboards = 10
tmp = features_acts_by_token_df.sort_values(UNIQUE_TOKEN_INTEREST, ascending=False).T
# dashboard_features = features_acts_by_token_df.sort_values(UNIQUE_TOKEN_INTEREST, ascending=False).index[:n_feature_dashboards].to_series().apply(lambda x: x.split("_")[1]).tolist()
# for feature in dashboard_features:
#     render_feature_dashboard(feature)

px.line(tmp + 1e-5,
        # log_y=True,
        title=f"{title}: Features Activation by Token in Prompt", 
        color_discrete_sequence=px.colors.qualitative.Plotly,
        height=500).show()

tmp = features_acts_by_token_df.head(100).T
px.imshow(tmp, 
            title=f"{title}: Top k features by activation", 
            color_continuous_midpoint=0, 
            color_continuous_scale="RdBu", 
            height=800).show()

In [ ]:
render_feature_dashboard(17159)

# Interp Experiment

Let's contruct a bunch of cases where we have brackets.

In [ ]:
def generate_bracket_prompt():
    
    left_bracket_token = torch.tensor(model.to_single_token("(")).unsqueeze(0).repeat(1,256)
    right_bracket_token = torch.tensor(model.to_single_token(")")).unsqueeze(0).repeat(1,256)
    random_tokens = torch.randint(0, 50257, (10,256))
    
    # add the brackets after the first two tokens and before the last two tokensx
    prompt = torch.concat([random_tokens[:2], left_bracket_token, random_tokens[2:-2:], right_bracket_token, random_tokens[-2:]], dim=0).T
    return prompt

tokens = generate_bracket_prompt()
(original_logits, original_loss), original_cache = model.run_with_cache(tokens, return_type="both", loss_per_token=True)

original_act = original_cache[sparse_autoencoder.cfg.hook_point]
sae_out, feature_acts, _, mse_loss, _, _ = sparse_autoencoder(original_act)

In [ ]:
model.to_string(tokens[0])

## Show Feature Activation

In [ ]:
feature_of_interest= 17159
px.line(feature_acts[:,:,feature_of_interest].mean(0).T.detach().cpu()).show()
px.line(feature_acts[:,:,feature_of_interest].T.detach().cpu()).show()
w_dec_bracket_10 = sparse_autoencoder.W_dec[feature_of_interest].detach().cpu()

## DFA

In [ ]:
# DFA
decomp, labels = original_cache.decompose_resid(return_labels=True, layer = 10)
proj = decomp.detach().cpu() @ w_dec_bracket_10
proj_df = pd.DataFrame(proj.mean(1).T, columns=labels, index = range(proj.shape[2]))
# px.imshow(proj_df.T, title="Projection of Residuals onto Bracket Feature", color_continuous_midpoint=0, color_continuous_scale="RdBu")
px.line(proj_df.T, title="Projection of Residuals onto Bracket Feature", color_discrete_sequence=px.colors.qualitative.Plotly).show()

decomp, labels = original_cache.accumulated_resid(return_labels=True, layer = 10, incl_mid=True)

proj = decomp.detach().cpu() @ w_dec_bracket_10
proj_df = pd.DataFrame(proj.mean(1).T, columns=labels, index = range(proj.shape[2]))
# px.imshow(proj_df.T, title="Projection of Residuals onto Bracket Feature", color_continuous_midpoint=0, color_continuous_scale="RdBu")
px.line(proj_df.T, title="Projection of Residuals onto Bracket Feature (Accumulated)", color_discrete_sequence=px.colors.qualitative.Plotly)


In [ ]:
# DFA
decomp, labels = original_cache.get_full_resid_decomposition(return_labels=True, layer = 10, expand_neurons=False)
proj = decomp.detach().cpu() @ w_dec_bracket
proj_df = pd.DataFrame(proj.mean(1).T, columns=labels, index = range(proj.shape[2]))
# px.imshow(proj_df.T, title="Projection of Residuals onto Bracket Feature", color_continuous_midpoint=0, color_continuous_scale="RdBu")
px.line(proj_df.iloc[:,:12].T, title="Projection of Residuals onto Bracket Feature", color_discrete_sequence=px.colors.qualitative.Plotly).show()



## Attn

In [ ]:
px.imshow(
    original_cache['blocks.0.attn.hook_pattern'].mean(0).detach().cpu(),
    facet_col=0,
    facet_col_wrap=4,
    title="Attention Patterns",
    color_continuous_scale="RdBu",
    color_continuous_midpoint=0,
    height=800,
    width=800,
    # show all tics 
)

## Layer 1 DFA

In [ ]:
layer_1_sae_path = "../GPT2-small-SAEs/final_sparse_autoencoder_gpt2-small_blocks.1.hook_resid_pre_24576.pt"
sparse_autoencoder_layer_1 = SparseAutoencoder.load_from_pretrained(layer_1_sae_path)
sparse_autoencoder_layer_1.cfg.use_ghost_grads = False
original_act = original_cache[sparse_autoencoder_layer_1.cfg.hook_point]
sae_out, feature_acts_layer_1, _, mse_loss, _, _ = sparse_autoencoder_layer_1(original_act)

In [ ]:
feature_acts_layer_1.shape

In [ ]:
feature_of_interest= 11114
px.line(feature_acts_layer_1[:,:,feature_of_interest].mean(0).T.detach().cpu()).show()
px.line(feature_acts_layer_1[:,:,feature_of_interest].T.detach().cpu())

In [ ]:
# let's do DFA on this feature
w_dec_bracket_layer_1 = sparse_autoencoder_layer_1.W_dec[feature_of_interest].detach().cpu()

# DFA
decomp, labels = original_cache.get_full_resid_decomposition(return_labels=True, layer = 1, expand_neurons=False)
proj = decomp.detach().cpu() @ w_dec_bracket_layer_1
proj_df = pd.DataFrame(proj.mean(1).T, columns=labels, index = range(proj.shape[2]))
px.line(proj_df.T)

## Patching Experiment

In [ ]:
def generate_bracket_prompt_clean():
    
    left_bracket_token = torch.tensor(model.to_single_token("(")).unsqueeze(0).repeat(1,256)
    right_bracket_token = torch.tensor(model.to_single_token(")")).unsqueeze(0).repeat(1,256)
    random_tokens = torch.randint(0, 50257, (10,256))
    
    # add the brackets after the first two tokens and before the last two tokens
    prompt = torch.concat([random_tokens[:2], left_bracket_token, random_tokens[2:-2:], right_bracket_token, random_tokens[-2:]], dim=0).T
    return prompt

def generate_bracket_prompt_corrupt():
    
    left_bracket_token = torch.tensor(model.to_single_token("(")).unsqueeze(0).repeat(1,256)
    random_tokens = torch.randint(0, 50257, (11,256))
    
    # add the brackets after the first two tokens and before the last two tokens
    prompt = torch.concat([random_tokens[:2], left_bracket_token, random_tokens[2:]], dim=0).T
    return prompt


clean_tokens = generate_bracket_prompt_clean()
corrupted_tokens = generate_bracket_prompt_corrupt()

In [ ]:
_, clean_cache = model.run_with_cache(clean_tokens, return_type="both")
clean_act = original_cache[sparse_autoencoder.cfg.hook_point]
clean_sae_out, clean_feature_acts, _, mse_loss, _, _ = sparse_autoencoder(clean_act)

_, corrupt_cache = model.run_with_cache(corrupted_tokens, return_type="both")
corrupt_act = corrupt_cache[sparse_autoencoder.cfg.hook_point]
corrupt_sae_out, corrupt_feature_acts, _, mse_loss, _, _ = sparse_autoencoder(corrupt_act)

In [ ]:
feature_of_interest= 17159
px.line(
    y=[clean_feature_acts[:,:,feature_of_interest].mean(0).T.detach().cpu(), 
     corrupt_feature_acts[:,:,feature_of_interest].mean(0).T.detach().cpu()]).show()

Now I think that L0H7 seems critical, let's see if we patch the output for L0H7

In [ ]:

head_hook_result_name = utils.get_act_name("z", 0)
def hook_to_patch_head(
    head_output: Float[Tensor, "batch seq_len head_idx d_head"], 
    hook: HookPoint, 
    head = 7,
    pos = None,
    from_cache = clean_cache,):
    # print(head_output.shape)
    if pos is None:
        head_output[:, :, head[1], :] = from_cache[hook][:,:,head[1],:]
    else:
        head_output[:, pos, head[1], :] = from_cache[hook][:,pos,head[1],:]
    return head_output

with model.run_with_hooks(corrupted_tokens, fwd_hooks=[(head_hook_result_name, hook_to_patch_head)]):
    _, patched_cache = model.run_with_cache(corrupted_tokens, return_type="both")
    
    
    